Load ISL Translate Dataset

In [1]:
import torch
import numpy as np
from datasets import load_dataset

In [2]:
from huggingface_hub import login
from google.colab import userdata
login(userdata.get('hf_token'))

In [3]:
ds = load_dataset("Exploration-Lab/iSign", "iSign_v1.1", split='train')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.81k [00:00<?, ?B/s]

iSign_v1.1.csv:   0%|          | 0.00/9.81M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127237 [00:00<?, ? examples/s]

In [23]:
for i, sample in enumerate(ds.select(range(5))):
  print(sample)

{'uid': '1782bea75c7d-1', 'text': 'Page 111'}
{'uid': '1782bea75c7d-2', 'text': 'Make it shorter.'}
{'uid': '1782bea75c7d-3', 'text': 'One day, Akbar drew a line on the floor and ordered.'}
{'uid': '1782bea75c7d-4', 'text': 'Make this line shorter.'}
{'uid': '1782bea75c7d-5', 'text': "but don't rub out any part of it."}


Dataset Pipeline

In [5]:
from torch.utils.data import Dataset

In [6]:
class_to_label = {}

In [31]:
class ISLTranslateDataset(Dataset):
    def __init__(self, data_list):
        self.samples = data_list

        # self.cache = {}
        # self.cache = dict(enumerate(load_saved_data(saved_data_path, train)))

        # for sample in data_list:
        #   # num_augments = max(1, max_augments // num_files[file[1]])

        #   for _ in range(num_augments):
        #     self.video_paths.append(file[0])
        #     self.labels.append(file[1])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video, label = self.samples[idx].values()
        return video, label

def video_collate_fn(batch):
  batch = list(filter(lambda x: x[1] is not None, batch))

  videos, labels = zip(*batch)
  max_size = max(video.shape[0] for video in videos)

  padded_videos = []
  for video in videos:
    pad_size = max_size - video.shape[0]
    pad_tensor = torch.zeros((pad_size, *video.shape[1:]))
    padded_videos.append(torch.cat([pad_tensor, video], dim=0))

  labels = [class_to_label[label] for label in labels]

  return torch.stack(padded_videos), torch.tensor(labels)

In [32]:
train_data = ISLTranslateDataset(ds)

In [33]:
len(train_data)

127237

In [36]:
train_data[1]

('1782bea75c7d-2', 'Make it shorter.')